### MSDS 453 - Final Project
Summer 2024
Akhilesh Nair

Import Necessary Libraries

In [18]:
import tensorflow
import pandas as pd
import numpy
import sklearn
import transformers
import torch
import re

Load in google dataset: GeoEmotions

In [16]:
dataset1 = pd.read_csv('https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv')
dataset2 = pd.read_csv('https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv')
dataset3 = pd.read_csv('https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_3.csv')

frame = [dataset1, dataset2, dataset3]
full_data = pd.concat(frame)

In [17]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 211225 entries, 0 to 71224
Data columns (total 37 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   text                  211225 non-null  object 
 1   id                    211225 non-null  object 
 2   author                211225 non-null  object 
 3   subreddit             211225 non-null  object 
 4   link_id               211225 non-null  object 
 5   parent_id             211225 non-null  object 
 6   created_utc           211225 non-null  float64
 7   rater_id              211225 non-null  int64  
 8   example_very_unclear  211225 non-null  bool   
 9   admiration            211225 non-null  int64  
 10  amusement             211225 non-null  int64  
 11  anger                 211225 non-null  int64  
 12  annoyance             211225 non-null  int64  
 13  approval              211225 non-null  int64  
 14  caring                211225 non-null  int64  
 15  confus

Cleaning the text! (I know it's given to me by google but I want to do it myself)

In [19]:
# function to convert text to lowercase while also removing special char, nums and punctuation
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

full_data['cleaned_text'] = full_data['text'].apply(clean_text)

Let's now drop all the useless columns, like the author, time created, and rater approval